In [ ]:
# This code run over all files present in google colab folder and finds the fitted ellipse parameter and stores in csv file

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.image import imread
from matplotlib.patches import Ellipse
import pandas as pd
import glob
def print_image(pixel_intensity, cmap, title):
    plt.imshow(pixel_intensity, cmap=cmap)
    plt.axis('off')
    plt.title(title)
    plt.show()

def perform_dilation(edges, no_of_interation):
    kernel = np.ones((4,4), np.uint8)
    return cv2.dilate(edges, kernel, iterations=1)

def fit_ellipse(max_contour):
    ellipse = cv2.fitEllipse(max_contour)
    xc, yc = ellipse[0]
    a = max(ellipse[1][0] / 2, ellipse[1][1] / 2)
    b = min(ellipse[1][1] / 2, ellipse[1][0] / 2)
    theta = np.deg2rad(ellipse[2])

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return a, b, xc, yc, theta

def find_contours(edges):
    _, binary_edges = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)
    return contour

def find_approx_contours(contour):
    epsilon = 0.00001 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    return approx

def print_apple_image_with_boundary_points(image, boundary_points, apple_label):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), origin='lower')
    plt.scatter(*zip(*boundary_points), color='green', s=1)
    plt.gca().invert_yaxis()  # Invert y-axis to match image coordinates
    plt.gca().set_aspect('equal', adjustable='box')  # Set aspect ratio to be equal
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.title(f'Boundary Points of the Apple {apple_label}')
    plt.show()

def yellow_to_red_pixels(image):
    lower_yellow = np.array([100, 100, 0])  # Lower bound for spread yellow range (decreased)
    upper_yellow = np.array([255, 255, 150])  # Upper bound for spread yellow range (increased)
    lower_red = np.array([100, 0, 0])  # Lower bound for moderate/less red range
    upper_red = np.array([255, 50, 50])  # Upper bound for moderate/less red range
    lower_white = np.array([200, 200, 200])  # Lower bound for low/moderate white range (increased)

    yellow_mask = np.all((image >= lower_yellow) & (image <= upper_yellow), axis=-1)
    red_mask = np.all((image >= lower_red) & (image <= upper_red), axis=-1)
    white_mask = np.all(image >= lower_white, axis=-1)

    # Convert yellow pixels to more red
    image_copy = np.copy(image)
    image_copy[yellow_mask] = [255, 0, 0]  # More red

    # Convert moderate/less red pixels to high red
    image_copy[red_mask] = [255, 0, 0]  # High red

    # Convert low/moderate white pixels to complete white
    image_copy[white_mask] = [255, 255, 255]  # Complete white

    # plt.imshow(image_copy)
    # plt.axis('off')
    # plt.show()
    return image_copy

# Function to find the boundary points of the apple in the image
def find_countour_and_boundary_points(image_path):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image with guassian kernal of 11*11, higher the kernal size that blurring effect will be more pronounced
    blurred = cv2.GaussianBlur(gray, (13, 13), 0)
    # print_image(blurred, 'gray', 'Blurred Image')

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    # print_image(edges, 'gray', 'Canny Edges')

    # kernal for dilation
    kernel = np.ones((3,3), np.uint8)
    edges = perform_dilation(edges, 1)
    # print_image(edges, 'gray', 'Canny Edges After Dilation')

    contours = find_contours(edges)
    approx = find_approx_contours(contours)
    # Convert the boundary points to a list of tuples
    boundary_points = [tuple(point[0]) for point in approx]
    return contours, boundary_points

def get_other_ellipse_parameters(a, b, xc, yc, theta):
    # Calculate area of the ellipse
    area = np.pi * a * b
    # Calculate perimeter of the ellipse (approximation)
    perimeter = np.pi * (3 * (a + b) - np.sqrt((3 * a + b) * (a + 3 * b)))
    # Calculate eccentricity of the ellipse
    eccentricity = np.sqrt(1 - (b**2 / a**2))
    return area, perimeter, eccentricity

ellipse_parameters = []
# Define the directory path without escaping the spaces
directory = '/content/drive/My Drive/Apple_Volume_Estimation_Images_Cropped'
# Use glob to find all files in the directory
files = glob.glob(os.path.join(directory, '*'))
# Iterate over the files
count = 0
for file in files:
    image_path = file
    image = imread(image_path)
    filename = image_path[63:]
    image = yellow_to_red_pixels(image)
    contours, boundary_points = find_countour_and_boundary_points(image_path)
    a, b, xc, yc, theta = fit_ellipse(contours)
    area, perimeter, eccentricity = get_other_ellipse_parameters(a, b, xc, yc, theta)
    ellipse_instance_parameter = [filename, a, b, xc, yc, theta, area, perimeter, eccentricity]
    print(ellipse_instance_parameter)
    ellipse_parameters.append(ellipse_instance_parameter)
    image = cv2.imread(image_path)
    count = count + 1
    
    ellipse_parameters = sorted(ellipse_parameters, key=lambda x: x[0])
    df = pd.DataFrame(ellipse_parameters, columns=['apple_label', 'semi_major_axis', 'semi_minor_axis', 'xc', 'yc', 'theta', 'area', 'perimeter', 'eccentricity'])
    df.to_csv('fitted_ellipse_parameter.csv', index=False, float_format='%.2f')
    print(count)
